<a href="https://colab.research.google.com/github/mangohehe/rags/blob/main/Rag_with_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG System Using Llama2 With Hugging Face

In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
!pip install -U sentence-transformers

In [ ]:
!pip install --upgrade llama-index

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate

In [ ]:
!pip install llama-index-llms-huggingface
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

In [ ]:
system_prompt="""
You are a Job Seeking Advisor Assistant. Your primary role is to provide accurate and detailed advice on the skills required for specific job positions, based on the job descriptions and context provided. You should also guide users on how they can improve or acquire these skills to enhance their employability.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("{query_str}")

In [ ]:
!huggingface-cli login

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
service_context

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is attention is all you need?")

In [ ]:
print(response)

In [ ]:
response=query_engine.query("what Machine Learning Software Engineer skills are required?")

In [ ]:
print(response)

In [ ]:
print(response)